# 摘要
在這裏我要用 MovieLens 資料集做一個 item-based 協同式過濾 (collaborative filtering) 系統。我會基於使用者的的評分，把電影分群，建立一個 KNN 模型，然後使用這個模型推薦類似評分的電影

## 推薦系統 Recommender system
推薦系統基本上就是一個資訊過濾系統。這個系統的終極目的是預測一個使用者對於一件產品的評分或是喜好。推薦系統廣泛用於各類型的網路服務上，例如串流媒體 Netflix, Spotify, 電商 Amazon, PChome, 社群媒體 YouTube, Facebook. 這些公司或服務透過推薦系統來提供個人化的產品和服務來達到盈利最大化。

推薦系統通常會透過協同式過濾 collaborative filtering 或是內容導向e過濾 Content based filtering 來產生推薦清單。這篇會先把重心放在物品導向的協同式過濾系 item-based collaborative filtring 統來產生電影推薦。

## 物品導向協同式過濾 Item-based Collaborative filtering
協同式過濾系統透過使用者的行為來推薦物品，大致上來說協同式過濾又可以分為使用者導向或是物品導向兩種方式。使用者協同過濾透過找出類似的使用者產生推薦(與你類似的使用者也在看這些商品)。物品協同式過濾透過瀏覽物品相同的使用者產生推薦(瀏覽這項商品的使用者也看了這些商品)。 因為人類的行為相較於物品動態幅度來的高，物品協統式過濾通常會先被採用作為初步的推薦系統。

## 資料集


In [2]:
import os
import time

# data science imports
import pandas as pd
import math
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# utils import
from fuzzywuzzy import fuzz

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# read data
df_movies = pd.read_csv('../movies.csv', usecols=['movieId', 'title'], dtype={'movieId':'int32', 'titile':'str'})
df_ratings = pd.read_csv('../ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating':'float32'})

In [5]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [6]:
df_ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
